In [1]:
from __future__ import annotations
from dataclasses import dataclass
from collections import Counter
from pathlib import Path
import math
import random
import time
from typing import Dict, Tuple, Optional, Any

import numpy as np
import pandas as pd

# ---------------------------------------------------------------------------
# 0.  Generic helpers
# ---------------------------------------------------------------------------

def _entropy(labels) -> float:
    counts = Counter(labels)
    total = len(labels)
    return -sum((c / total) * math.log2(c / total) for c in counts.values())


def _gini(labels) -> float:
    counts = Counter(labels)
    total = len(labels)
    return 1.0 - sum((c / total) ** 2 for c in counts.values())

# ---------------------------------------------------------------------------
# 1.  Numeric-feature threshold search & discretisers
# ---------------------------------------------------------------------------

def _best_threshold(values: np.ndarray, labels: np.ndarray, *, criterion: str = "entropy") -> Tuple[Optional[float], float]:
    order = np.argsort(values)
    v_sorted, y_sorted = values[order], labels[order]
    uniques = np.unique(v_sorted)
    if len(uniques) == 1:
        return None, 0.0

    impurity = _entropy if criterion == "entropy" else _gini
    total_imp = impurity(y_sorted)

    best_gain, best_thr = 0.0, None
    n = len(values)
    for i in range(1, n):
        if v_sorted[i] == v_sorted[i - 1]:
            continue
        thr = (v_sorted[i] + v_sorted[i - 1]) / 2
        left_y, right_y = y_sorted[:i], y_sorted[i:]
        gain = total_imp - (
            (len(left_y) / n) * impurity(left_y) + (len(right_y) / n) * impurity(right_y)
        )
        if gain > best_gain:
            best_gain, best_thr = gain, thr
    return best_thr, best_gain


def discretise_equal_width(series: pd.Series, bins: int = 4) -> pd.Series:
    return pd.cut(series, bins=bins, labels=False, include_lowest=True)


def discretise_entropy(series: pd.Series, labels: pd.Series):
    thr, _ = _best_threshold(series.to_numpy(), labels.to_numpy(), criterion="entropy")
    if thr is None:
        return pd.Series(np.zeros(len(series), dtype=int), index=series.index), None
    return (series <= thr).astype(int), thr


def apply_binary_entropy(X: pd.DataFrame, y: pd.Series) -> Tuple[pd.DataFrame, Dict[str, float]]:
    Xd, thresholds = {}, {}
    for col in X.columns:
        if np.issubdtype(X[col].dtype, np.number):
            Xd[col], thr = discretise_entropy(X[col], y)
            if thr is not None:
                thresholds[col] = thr
        else:
            Xd[col] = X[col]
    return pd.DataFrame(Xd), thresholds

# ---------------------------------------------------------------------------
# 2.  Shared node dataclass
# ---------------------------------------------------------------------------

@dataclass
class Node:
    prediction: Any
    feature: Optional[str] = None
    threshold: Optional[float] = None
    children: Dict[Any, "Node"] | None = None
    left: "Node" | None = None
    right: "Node" | None = None

    def is_leaf(self):
        return self.feature is None

# ---------------------------------------------------------------------------
# 3.  ID3 implementation (categorical-only)
# ---------------------------------------------------------------------------

class ID3DecisionTree:
    def print_tree(self, node: Optional[Node] = None, indent: str = "", depth: int = 0, max_depth_print: int = 2):
        if node is None:
            node = self.root
        if node.is_leaf() or depth >= max_depth_print:
            print(indent + f"Label: {node.prediction}")
            return
        print(indent + f"Feature: {node.feature}")
        for val, child in node.children.items():
            print(indent + f" ├── [{val}]")
            self.print_tree(child, indent + " │   ", depth + 1, max_depth_print)

    def __init__(self, max_depth: Optional[int] = None):
        self.root: Node | None = None
        self.max_depth = max_depth

    def _info_gain(self, data: pd.DataFrame, feature: str) -> float:
        total = _entropy(data["label"])
        weighted = sum(
            len(sub) / len(data) * _entropy(sub["label"]) for _, sub in data.groupby(feature)
        )
        return total - weighted

    def _build(self, data: pd.DataFrame, features: pd.Index, depth: int) -> Node:
        y = data["label"]
        if len(y.unique()) == 1:
            return Node(prediction=y.iloc[0])
        if not len(features) or (self.max_depth is not None and depth >= self.max_depth):
            return Node(prediction=y.mode()[0])

        best_feat = max(features, key=lambda f: self._info_gain(data, f))
        node = Node(prediction=y.mode()[0], feature=best_feat, children={})
        for val, sub in data.groupby(best_feat):
            node.children[val] = self._build(sub.drop(columns=[best_feat]), features.drop(best_feat), depth + 1)
        return node

    def fit(self, X: pd.DataFrame, y: pd.Series):
        data = X.copy()
        data["label"] = y
        self.root = self._build(data, X.columns, 0)

    def predict(self, X: pd.DataFrame):
        return X.apply(lambda r: self._predict_row(r, self.root), axis=1)

    def _predict_row(self, row, node):
        while not node.is_leaf():
            node = node.children.get(row[node.feature], node)
        return node.prediction

# ---------------------------------------------------------------------------
# 4.  CART implementation (numeric & categorical)
# ---------------------------------------------------------------------------

class CARTDecisionTree:
    def __init__(self, *, max_depth: Optional[int] = None, min_samples_split: int = 2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root: Node | None = None

    def print_tree(self, node: Optional[Node] = None, indent: str = "", depth: int = 0, max_depth_print: int = 2):
        if node is None:
            node = self.root
        if node.is_leaf() or depth >= max_depth_print:
            print(indent + f"Label: {node.prediction}")
            return
        # print numeric split
        print(indent + f"Feature: {node.feature} <= {node.threshold:.3f} ?")
        print(indent + " ├── True:")
        self.print_tree(node.left, indent + " │   ", depth+1, max_depth_print)
        print(indent + " └── False:")
        self.print_tree(node.right, indent + "     ", depth+1, max_depth_print)

    def _best_split(self, X: pd.DataFrame, y: pd.Series):
        best_gain, best_feat, best_thr, best_masks = 0.0, None, None, None
        base_imp = _gini(y)
        for feature in X.columns:
            col = X[feature]
            if np.issubdtype(col.dtype, np.number):
                thr, gain = _best_threshold(col.to_numpy(), y.to_numpy(), criterion="gini")
                if thr is None or gain <= best_gain:
                    continue
                mask = col <= thr
                best_gain, best_feat, best_thr, best_masks = gain, feature, thr, (mask, ~mask)
            else:
                for val in col.unique():
                    mask = col == val
                    gain = base_imp - (mask.mean() * _gini(y[mask]) + (~mask).mean() * _gini(y[~mask]))
                    if gain > best_gain:
                        best_gain, best_feat, best_thr, best_masks = gain, feature, val, (mask, ~mask)
        return best_feat, best_thr, best_masks, best_gain

    def _build(self, X: pd.DataFrame, y: pd.Series, depth: int) -> Node:
        if (
            len(y.unique()) == 1
            or len(X) < self.min_samples_split
            or (self.max_depth is not None and depth >= self.max_depth)
        ):
            return Node(prediction=y.mode()[0])
        feat, thr, masks, gain = self._best_split(X, y)
        if feat is None or gain == 0:
            return Node(prediction=y.mode()[0])
        node = Node(prediction=y.mode()[0], feature=feat, threshold=thr)
        left_mask, right_mask = masks
        node.left = self._build(X[left_mask], y[left_mask], depth + 1)
        node.right = self._build(X[right_mask], y[right_mask], depth + 1)
        return node

    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.root = self._build(X.reset_index(drop=True), y.reset_index(drop=True), 0)

    def predict(self, X: pd.DataFrame):
        return X.apply(lambda r: self._predict_row(r, self.root), axis=1)

    def _predict_row(self, row, node):
        while not node.is_leaf():
            if node.threshold is not None and np.issubdtype(type(row[node.feature]), np.number):
                node = node.left if row[node.feature] <= node.threshold else node.right
            else:
                node = node.left if row[node.feature] == node.threshold else node.right
        return node.prediction

    def score(self, X: pd.DataFrame, y: pd.Series):
        return (X.apply(lambda r: self._predict_row(r, self.root), axis=1) == y).mean()

# ---------------------------------------------------------------------------
# 5.  Simple train/test split helper (no sklearn)
# ---------------------------------------------------------------------------

def simple_train_test_split(X: pd.DataFrame, y: pd.Series, *, test_ratio: float = 0.3, seed: int = 0):
    idx = X.sample(frac=1.0, random_state=seed).index
    cut = int(len(idx) * (1 - test_ratio))
    return X.loc[idx[:cut]], X.loc[idx[cut:]], y.loc[idx[:cut]], y.loc[idx[cut:]]

# ---------------------------------------------------------------------------
# 6.  Demo & sanity check
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    iris = pd.read_csv("iris.csv")
    X_raw = iris.iloc[:, 1:-1]
    y = iris.iloc[:, -1]

    def run_demo(method: str):
        if method == "equal_width":
            X_disc = X_raw.apply(discretise_equal_width)
            tree = ID3DecisionTree()
            X_tr, X_te, y_tr, y_te = simple_train_test_split(X_disc, y)
            tree.fit(X_tr, y_tr)
            acc = (tree.predict(X_te) == y_te).mean() * 100
            print(f"Discretisation method: {method}. Thresholds = {{}}")
            print(f"Test accuracy: {acc:.2f}%")
            print("=== Learned tree ===")
            tree.print_tree(max_depth_print=2)

        elif method == "entropy":
            X_disc, thresholds = apply_binary_entropy(X_raw, y)
            tree = ID3DecisionTree()
            X_tr, X_te, y_tr, y_te = simple_train_test_split(X_disc, y)
            tree.fit(X_tr, y_tr)
            acc = (tree.predict(X_te) == y_te).mean() * 100
            print(f"Discretisation method: {method}. Thresholds = {thresholds}")
            print(f"Test accuracy: {acc:.2f}%")
            print("=== Learned tree ===")
            tree.print_tree(max_depth_print=2)

        elif method == "cart":

            X_tr, X_te, y_tr, y_te = simple_train_test_split(X_raw, y)
            tree = CARTDecisionTree()
            tree.fit(X_tr, y_tr)

            acc = tree.score(X_te, y_te) * 100
            print(f"Discretisation method: {method}. Thresholds = {{}}")
            print(f"Test accuracy: {acc:.2f}%")
            print("=== Learned tree ===")
            tree.print_tree(max_depth_print=2)

        else:
            raise ValueError(f"Invalid method: {method}")

    # uso
    run_demo("equal_width")
    run_demo("entropy")
    run_demo("cart")


Discretisation method: equal_width. Thresholds = {}
Test accuracy: 95.56%
=== Learned tree ===
Feature: petalwidth
 ├── [0]
 │   Label: Iris-setosa
 ├── [1]
 │   Label: Iris-versicolor
 ├── [2]
 │   Feature: petallength
 │    ├── [1]
 │    │   Label: Iris-versicolor
 │    ├── [2]
 │    │   Label: Iris-versicolor
 │    ├── [3]
 │    │   Label: Iris-virginica
 ├── [3]
 │   Label: Iris-virginica
Discretisation method: entropy. Thresholds = {'sepallength': 5.55, 'sepalwidth': 3.3499999999999996, 'petallength': 2.45, 'petalwidth': 0.8}
Test accuracy: 68.89%
=== Learned tree ===
Feature: petallength
 ├── [0]
 │   Feature: sepallength
 │    ├── [0]
 │    │   Label: Iris-virginica
 │    ├── [1]
 │    │   Label: Iris-versicolor
 ├── [1]
 │   Label: Iris-setosa
Discretisation method: cart. Thresholds = {}
Test accuracy: 95.56%
=== Learned tree ===
Feature: petallength <= 2.450 ?
 ├── True:
 │   Label: Iris-setosa
 └── False:
     Feature: petalwidth <= 1.650 ?
      ├── True:
      │   Label: Ir

Equal-width bins (quartiles) kept 4 groups per measurement, so the tree had enough detail to tell the three iris species apart.

Entropy forced each measurement into just 2 groups (“small” or “large”).
With only two options, the tree lost the extra cut-point it needs to separate versicolor from virginica, so accuracy dropped.

Unlike binary discretisation, CART learns optimal split thresholds per feature at each node allowing multiple cut-points and preserving the granularity needed to separate similar classes